# Installing

In [ ]:
!pip install pyomo

In [ ]:
!pip install glpk

# Pyomo import

In [ ]:
from pyomo.environ import *

In [ ]:
import random

# Pyomo tutorial from documentation

# Models

In [ ]:
model = ConcreteModel()

In [ ]:
model = AbstractModel()

# Sets

In [ ]:
model.A = Set(initialize=['red', 'green', 'blue'])

In [ ]:
model.D = RangeSet(1.5, 10, 3.5)

In [ ]:
model.I = model.A | model.D # union
model.J = model.A & model.D # intersection
model.K = model.A - model.D # difference
model.L = model.A ^ model.D # exclusive-or

# within options:

Any = all possible values

Reals = floating point values

PositiveReals = strictly positive floating point values

NonPositiveReals = non-positive floating point values

NegativeReals = strictly negative floating point values

NonNegativeReals = non-negative floating point values

PercentFraction = floating point values in the interval [0,1]

UnitInterval = alias for PercentFraction

Integers = integer values

PositiveIntegers = positive integer values

NonPositiveIntegers = non-positive integer values

NegativeIntegers = negative integer values

NonNegativeIntegers = non-negative integer values

Boolean = Boolean values, which can be represented as False/True, 0/1, ’False’/’True’ and ’F’/’T’

Binary = same as Boolean

# Parameters

In [ ]:
model.A = Set()
model.B = Set()
model.P = Param(model.A, model.B)

In [ ]:
model.const_A = Param(initialize = -1, mutable=True)
model.const_B = Param(initialize = -1, mutable=True)

In [ ]:
model.const_A.set_value(2)
model.const_B.set_value(3)

# Variables

In [ ]:
model.LumberJack = Var(within=NonNegativeReals, bounds=(0,6), initialize=1.5)

# Objective

In [ ]:
def ObjRule(model):
    return 2*model.x[1] + 3*model.x[2]
model.Obj = Objective(rule=ObjRule, sense=maximize)

# Constraints

In [ ]:
model.x = Var()

def aRule(model):
    return model.x >= 2
model.Boundx = Constraint(rule=aRule)

def bRule(model):
    return (2, model.x, None)
model.boundx = Constraint(rule=bRule)

# Solving Pyomo Models

In [ ]:
!pyomo help --solvers

In [ ]:
def solve_model(modelo, parametros):
    opt = SolverFactory('glpk')
    sol = opt.solve(model) 
    return modelo

# 1. The Transport Problem

## Create a model

In [ ]:
# Creation of a Concrete Model
model = ConcreteModel()

## Set definitions

In [ ]:
model.i = Set(initialize=['seattle','san-diego'], doc='Canning plans')
model.j = Set(initialize=['new-york','chicago', 'topeka'], doc='Markets')

## Parameters

In [ ]:
model.a = Param(model.i, initialize={'seattle':350,'san-diego':600}, doc='Capacity of plant i in cases')
model.b = Param(model.j, initialize={'new-york':325,'chicago':300,'topeka':275}, doc='Demand at market j in cases')

In [ ]:
dtab = {
    ('seattle',  'new-york') : 2.5,
    ('seattle',  'chicago')  : 1.7,
    ('seattle',  'topeka')   : 1.8,
    ('san-diego','new-york') : 2.5,
    ('san-diego','chicago')  : 1.8,
    ('san-diego','topeka')   : 1.4,
    }
model.d = Param(model.i, model.j, initialize=dtab, doc='Distance in thousands of miles')

#  Scalar f  freight in dollars per case per thousand miles  /90/ ;
model.f = Param(initialize=90, doc='Freight in dollars per case per thousand miles')

In [ ]:
def c_init(model, i, j):
    return model.f * model.d[i,j] / 1000

model.c = Param(model.i, model.j, initialize=c_init, doc='Transport cost in thousands of dollar per case')

## Variables

In [ ]:
model.x = Var(model.i, model.j, bounds=(0.0,None), doc='Shipment quantities in case')

## Constraints

In [ ]:
# supply(i) .. sum (j, x(i,j)) =l= a(i)
def supply_rule(model, i):
    return sum(model.x[i,j] for j in model.j) <= model.a[i]

model.supply = Constraint(model.i, rule=supply_rule, doc='Observe supply limit at plant i')

# demand(j) .. sum(i, x(i,j)) =g= b(j);
def demand_rule(model, j):
    return sum(model.x[i,j] for i in model.i) >= model.b[j]  

model.demand = Constraint(model.j, rule=demand_rule, doc='Satisfy demand at market j')

## Objective function

In [ ]:
def objective_rule(model):
    return sum(model.c[i,j]*model.x[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

## Results

In [ ]:
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [ ]:
if __name__ == '__main__':
    # This emulates what the pyomo command-line tools does
    # pyomo solve –solver=glpk transport.py
    from pyomo.opt import SolverFactory
    import pyomo.environ
    opt = SolverFactory("glpk")
    results = opt.solve(model)
    #sends results to stdout
    results.write()
    print("\nDisplaying Solution\n" + '-'*60)
    pyomo_postprocess(None, model, results)

# 2. The p-Median Problem

In [ ]:
import random

random.seed(1000)

In [ ]:
model = AbstractModel()

In [ ]:
# Number of candidate locations
model.m = Param(within=PositiveIntegers)

# Number of customers
model.n = Param(within=PositiveIntegers)

# Set of candidate locations
model.M = RangeSet(1, model.m)

# Set of customer nodes
model.N = RangeSet(1, model.n)

# Number of facilities
model.p = Param(within=RangeSet(1,model.n))

# d[j] - demand of customer j
model.d = Param(model.N, default=1.0)

# c[i,j] - unit cost of satisfying customer j from facility i
model.c = Param(model.M, model.N, initialize=lambda i, j, model : random.uniform(1.0,2.0), within=Reals)

In [ ]:
# x[i,j] - fraction of the demand of customer j that is supplied by facility i
model.x = Var(model.M, model.N, bounds=(0.0, 1.0))

# y[i] - a binary value that is 1 is a facility is located at location i
model.y = Var(model.M, within=Binary)

In [ ]:
# Minimize the demand-weighted total cost
def cost_(model):
    return sum(model.d[j]*model.c[i,j]*model.x[i,j] for i in model.M for j in model.N)
model.cost = Objective(rule=cost_)

In [ ]:
# All of the demand for customer j must be satisfied
def demand_(model, j):
    return sum(model.x[i,j] for i in model.M) == 1.0
model.demand = Constraint(model.N, rule=demand_)

# Exactly p facilities are located
def facilities_(model):
    return sum(model.y[i] for i in model.M) == model.p
model.facilities = Constraint(rule=facilities_)

# Demand nodes can only be assigned to open facilities 
def openfac_(model, i, j):
    return model.x[i,j] <= model.y[i]
model.openfac = Constraint(model.M, model.N, rule=openfac_)

In [ ]:
instance = model.create_instance('p-median.dat')
opt = SolverFactory('glpk')
opt.solve(instance) 

# 3. Knapsack Problem

## Formulação matemática

$$ maximizar \sum_{i \in I}{x_i . v_i} $$

$$ \sum_{i \in I}{x_i . c_i} \leq C $$


In [ ]:
knapsack = ConcreteModel()

In [ ]:
knapsack.Items = Set(initialize=[i for i in range(10)], doc='itens disponíveis')

In [ ]:
def RandomRule(model, i):
    return random.random()

knapsack.Valores = Param(knapsack.Items, initialize=RandomRule, within=PositiveReals, mutable=False, doc='Valores dos itens')
knapsack.Custos = Param(knapsack.Items, initialize=RandomRule, within=PositiveReals, mutable=False, doc='Custo dos itens')
knapsack.Capacidade = Param(initialize=-1, mutable=True, doc='Capacidade total')

In [ ]:
knapsack.Valores.display()

In [ ]:
knapsack.x = Var(knapsack.Items, within=Binary, doc='variável de decisão binária indicando a escolha ou não de cada item')

In [ ]:
def ObjRule(model):
    return sum([model.x[i]*model.Valores[i] for i in model.Items])

knapsack.objetivo = Objective(rule=ObjRule, sense=maximize, doc='função objetivo: maximizar o valor dos itens selecionados')

In [ ]:
def RestCap(model):
    return sum([model.x[i]*model.Custos[i] for i in model.Items]) <= model.Capacidade

knapsack.total_cap = Constraint(rule=RestCap, doc='restrição para capacidade total')

In [ ]:
def solve_model(modelo, capacidade):
    modelo.Capacidade.set_value(capacidade)
    
    opt = SolverFactory('glpk')
    
    results = opt.solve(modelo)
    
    results.write()
    modelo.x.display()
    
    return modelo

In [ ]:
solve_model(knapsack, 2)